# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-NOTEBOOK_CODE_HERE-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)


# Politics Workflow Series ("Presidents of countries") 

Consider the following exploratory information need:

> You investigating presidents of the republic, or similar roles, across international states around the world

## Useful URIs for the current workflow


The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P279`    | subclass      | predicate |
| `wdt:P17`     | country       | predicate |
| `wdt:P27`     | citizenship   | predicate |
| `wdt:P39`     | position held   | predicate |
| `wd:Q248577`  | President of the republic | node      |
| `wd:Q11696`   | President of U.S.A.      | node      |
| `wd:Q332711`  | President of Italy        | node |
| `wd:Q38`      | Italy          | node |
| `wd:Q30`      | U.S.A.        | node |



Also consider

```
?p wdt:P39/wdt:P279* wd:Q248577  .  
```

is the BGP to retrieve all **presidents of the world countries through history**

## Workload Goals

1. Identify the BGP for obtaining important attributes for people, e.g., date of birth/death, gender, profession 

2. Identify the BGP to retrieve countries with  had at least once had a president

3. When was the first president of each country born? How many presidents each country had?

4. Is there a country that had at some point a woman as a president?

5. Analyze the number of presidents per country through time
 
   5.1 What are the top-5 countries for number of presidents? Which countries had the least?
   
   5.2 For how many presidents we know a date of death in each country?
   
   5.3 Which are the professions of different presidents? How many presidents had a specific profession?


In [2]:
# start your workflow here

In [4]:
queryString = """
SELECT COUNT(*)
WHERE { 

?p wdt:P39/wdt:P279* wd:Q248577  . 
} 
"""

print("Results")
run_query(queryString)

Results
[('callret-0', '1649')]


1

# Workload 1

In [8]:
# 1. Identify the BGP for obtaining important attributes for people, e.g., date of birth/death, gender, profession

In [9]:
# Find all the presidents

In [10]:
queryString = """
SELECT ?president ?presidentname
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  . 

?president <http://schema.org/name> ?presidentname .

} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('president', 'http://www.wikidata.org/entity/Q14345'), ('presidentname', 'Roh Moo-hyun')]
[('president', 'http://www.wikidata.org/entity/Q472323'), ('presidentname', 'Ranasinghe Premadasa')]
[('president', 'http://www.wikidata.org/entity/Q193062'), ('presidentname', 'Raúl Alfonsín')]
[('president', 'http://www.wikidata.org/entity/Q382830'), ('presidentname', 'Delfim Moreira')]
[('president', 'http://www.wikidata.org/entity/Q81215'), ('presidentname', 'Petar Stoyanov')]
[('president', 'http://www.wikidata.org/entity/Q3514836'), ('presidentname', 'Tammam Salam')]
[('president', 'http://www.wikidata.org/entity/Q15029'), ('presidentname', 'Hu Jintao')]
[('president', 'http://www.wikidata.org/entity/Q206479'), ('presidentname', 'Émile Lahoud')]
[('president', 'http://www.wikidata.org/entity/Q57478'), ('presidentname', 'Nicolae Nasoc')]
[('president', 'http://www.wikidata.org/entity/Q57634'), ('presidentname', 'Hashim Thaçi')]


10

In [ ]:
# find president instance of ... to select human president 

In [142]:
queryString = """
SELECT ?president ?presidentname ?s ?sname
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  ; 
           wdt:P31 ?s .

?president <http://schema.org/name> ?presidentname .
?s <http://schema.org/name> ?sname .
} 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('president', 'http://www.wikidata.org/entity/Q3545001'), ('presidentname', 'Gonzo the Mechanical Bastard'), ('s', 'http://www.wikidata.org/entity/Q1114461'), ('sname', 'comics character')]
[('president', 'http://www.wikidata.org/entity/Q5297058'), ('presidentname', 'Doom 2099'), ('s', 'http://www.wikidata.org/entity/Q1114461'), ('sname', 'comics character')]
[('president', 'http://www.wikidata.org/entity/Q4003185'), ('presidentname', 'Ultimate Thor'), ('s', 'http://www.wikidata.org/entity/Q1114461'), ('sname', 'comics character')]
[('president', 'http://www.wikidata.org/entity/Q7880141'), ('presidentname', 'Ultimate Captain America'), ('s', 'http://www.wikidata.org/entity/Q1114461'), ('sname', 'comics character')]
[('president', 'http://www.wikidata.org/entity/Q7242478'), ('presidentname', 'Prez'), ('s', 'http://www.wikidata.org/entity/Q1114461'), ('sname', 'comics character')]
[('president', 'http://www.wikidata.org/entity/Q3545001'), ('presidentname', 'Gonzo the Mechanical 

20

In [ ]:
# Look above I notice that there are a lot of president instance of 'fictional human' or 'comics character' or 'film character'
# I think it's better to 'delete' these president, to investigate better.
# GET: wd:Q5 --> humman

In [29]:
# Find attributes for presidents 

In [30]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  ;
            ?p ?o .

#?president <http://schema.org/name> ?presidentname .
?p <http://schema.org/name> ?pname .
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P7691'), ('pname', 'The Oregon Encyclopedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P4026'), ('pname', 'Cullum number')]
[('p', 'http://www.wikidata.org/prop/direct/P1006'), ('pname', 'Nationale Thesaurus voor Auteurs ID')]
[('p', 'http://www.wikidata.org/prop/direct/P101'), ('pname', 'field of work')]
[('p', 'http://www.wikidata.org/prop/direct/P1015'), ('pname', 'NORAF ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1017'), ('pname', 'Vatican Library ID (former scheme)')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('pname', 'member of political party')]
[('p', 'http://www.wikidata.org/prop/direct/P103'), ('pname', 'native language')]
[('p', 'http://www.wikidata.org/prop/direct/P1038'), ('pname', 'relative')]
[('p', 'http://www.wikidata.org/prop/direct/P1048'), ('pname', 'NCL ID')]


10

In [31]:
# Look for date of birthday/death

In [32]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  ;
            ?p ?o .

#?president <http://schema.org/name> ?presidentname .
?p <http://schema.org/name> ?pname .

FILTER(REGEX(?pname, "date"))
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P569'), ('pname', 'date of birth')]
[('p', 'http://www.wikidata.org/prop/direct/P570'), ('pname', 'date of death')]
[('p', 'http://www.wikidata.org/prop/direct/P4602'), ('pname', 'date of burial or cremation')]
[('p', 'http://www.wikidata.org/prop/direct/P8748'), ('pname', 'Rheinland-Pfälzische Personendatenbank ID')]
[('p', 'http://www.wikidata.org/prop/direct/P576'), ('pname', 'dissolved, abolished or demolished date')]
[('p', 'http://www.wikidata.org/prop/direct/P1636'), ('pname', 'date of baptism in early childhood')]


6

In [33]:
# GET:
# date of birth --> wdt:P569
# date of death --> wdt:P570

In [34]:
# Look for gender

In [35]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  ;
            ?p ?o .

#?president <http://schema.org/name> ?presidentname .
?p <http://schema.org/name> ?pname .

FILTER(REGEX(?pname, "gender"))
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P21'), ('pname', 'sex or gender')]


1

In [ ]:
# GET: 
# sex or gender --> wdt:P21

In [36]:
# Look for occupation 

In [39]:
queryString = """
SELECT DISTINCT ?p ?pname
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577  ;
            ?p ?o .

#?president <http://schema.org/name> ?presidentname .
?p <http://schema.org/name> ?pname .

FILTER(REGEX(?pname, "occupation"))
} 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P106'), ('pname', 'occupation')]


1

In [ ]:
# GET: 
# occupation --> wdt:P106

In [148]:
queryString = """
SELECT DISTINCT ?president ?presidentname ?dateofbirth ?dateofdeath ?sexorgender ?occupation
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577 .
OPTIONAL {          
?president wdt:P569 ?dateofbirth ;
           wdt:P570 ?dateofdeath ;
           wdt:P21  ?sex         ;
           wdt:P31  wd:Q5        ;
           wdt:P106 ?oc          
            } .

?president <http://schema.org/name> ?presidentname .
?sex <http://schema.org/name> ?sexorgender .
?oc <http://schema.org/name> ?occupation .

FILTER REGEX(?presidentname, "Napolita")
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
Empty


0

In [52]:
# I want to find all president alive 
# I think to find italian presidents to see their attributes.. for example 'Napolitano'

In [149]:
queryString = """
SELECT DISTINCT ?president ?presidentname ?dateofbirth ?dateofdeath ?sexorgender ?occupation
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577 .
OPTIONAL {          
?president wdt:P569 ?dateofbirth ;
           wdt:P570 ?dateofdeath ;
           wdt:P21  ?sex         ;
           wdt:P31  wd:Q5        ;
           wdt:P106 ?oc } .

?president <http://schema.org/name> ?presidentname .
?sex <http://schema.org/name> ?sexorgender .
?oc <http://schema.org/name> ?occupation .

FILTER REGEX(?presidentname, "Napolita")
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
Empty


0

In [35]:
# Looking above I notice that the query to find  'Napolitano' president it's wrong beacuse this BGP gives me 
# only death presidents.. so I have to change it...

In [150]:
queryString = """
SELECT DISTINCT ?president ?presidentname ?dateofbirth ?dateofdeath ?sexorgender ?occupation
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577 .
OPTIONAL {          
?president wdt:P569 ?dateofbirth ;
           wdt:P21  ?sex         ;
           wdt:P31  wd:Q5        ;
           wdt:P106 ?oc } .
           
           OPTIONAL {
           ?president wdt:P570 ?dateofdeath 
           } .

?president <http://schema.org/name> ?presidentname .
?sex <http://schema.org/name> ?sexorgender .
?oc <http://schema.org/name> ?occupation .
 
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('president', 'http://www.wikidata.org/entity/Q14345'), ('presidentname', 'Roh Moo-hyun'), ('dateofbirth', '1946-08-06T00:00:00Z'), ('dateofdeath', '2009-05-23T00:00:00Z'), ('sexorgender', 'male'), ('occupation', 'judge')]
[('president', 'http://www.wikidata.org/entity/Q14345'), ('presidentname', 'Roh Moo-hyun'), ('dateofbirth', '1946-08-06T00:00:00Z'), ('dateofdeath', '2009-05-23T00:00:00Z'), ('sexorgender', 'male'), ('occupation', 'lawyer')]
[('president', 'http://www.wikidata.org/entity/Q14345'), ('presidentname', 'Roh Moo-hyun'), ('dateofbirth', '1946-08-06T00:00:00Z'), ('dateofdeath', '2009-05-23T00:00:00Z'), ('sexorgender', 'male'), ('occupation', 'politician')]
[('president', 'http://www.wikidata.org/entity/Q472323'), ('presidentname', 'Ranasinghe Premadasa'), ('dateofbirth', '1924-06-23T00:00:00Z'), ('dateofdeath', '1993-05-01T00:00:00Z'), ('sexorgender', 'male'), ('occupation', 'politician')]
[('president', 'http://www.wikidata.org/entity/Q193062'), ('presidentname', 

10

In [36]:
# Now it works! 

In [ ]:
# I try to find now 'Napolitano'

In [151]:
queryString = """
SELECT DISTINCT ?president ?presidentname ?dateofbirth ?dateofdeath ?sexorgender ?occupation
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577 .
OPTIONAL {          
?president wdt:P569 ?dateofbirth ;
          # wdt:P570 ?dateofdeath ;
           wdt:P21  ?sex         ;
           wdt:P31  wd:Q5        ;
           wdt:P106 ?oc } .
           
           OPTIONAL {
           ?president wdt:P570 ?dateofdeath 
           } .

?president <http://schema.org/name> ?presidentname .
?sex <http://schema.org/name> ?sexorgender .
?oc <http://schema.org/name> ?occupation .
 
 FILTER REGEX(?presidentname, "Napolita")
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('president', 'http://www.wikidata.org/entity/Q1220'), ('presidentname', 'Giorgio Napolitano'), ('dateofbirth', '1925-06-29T00:00:00Z'), ('sexorgender', 'male'), ('occupation', 'politician')]


1

In [ ]:
#
#

# Workload 2

In [ ]:
# 2. Identify the BGP to retrieve countries with had at least once had a president

In [ ]:
# We can start finding all the republics

In [4]:
queryString = """
SELECT ?rc ?republicountry
WHERE { 

?rc wdt:P279 wd:Q248577  . 

?rc <http://schema.org/name> ?republicountry .
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('rc', 'http://www.wikidata.org/entity/Q14946265'), ('republicountry', 'President of Bulgaria')]
[('rc', 'http://www.wikidata.org/entity/Q1054799'), ('republicountry', 'President of the Republic of Poland')]
[('rc', 'http://www.wikidata.org/entity/Q1625960'), ('republicountry', 'President of Albania')]
[('rc', 'http://www.wikidata.org/entity/Q18204367'), ('republicountry', 'President of Uganda')]
[('rc', 'http://www.wikidata.org/entity/Q19161338'), ('republicountry', 'President of Madagascar')]


5

In [ ]:
# Each republic is linked to a Country.. so i find it

In [ ]:
queryString = """ 
SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

In [ ]:
# I look for the presidents of each country counting it
# In this way I have all the republic of the world from 1 president to x

In [14]:
queryString = """ 
SELECT DISTINCT ?rcountry ?countryname (count(?pr) as ?num_president)
WHERE { 

?pr wdt:P39 ?rc ;         # ?pr = president
    wdt:P31  wd:Q5   .


{
 SELECT ?rc ?country 
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .


} 
}

    
?rc <http://schema.org/name> ?rcountry .

?country <http://schema.org/name> ?countryname .

} 
ORDER BY ASC (?num_president)
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('rcountry', 'President of the Republic of Puerto Rico'), ('countryname', 'Spanish Empire'), ('num_president', '1')]
[('rcountry', 'President of the Confederate States'), ('countryname', 'Confederate States of America'), ('num_president', '1')]
[('rcountry', 'President of the Republic of Kosova'), ('countryname', 'Republic of Kosova'), ('num_president', '1')]
[('rcountry', 'President of Turkmenistan'), ('countryname', 'Turkmenistan'), ('num_president', '2')]
[('rcountry', 'President of Kazakhstan'), ('countryname', 'Kazakhstan'), ('num_president', '2')]
[('rcountry', 'President of Serbia and Montenegro'), ('countryname', 'Serbia and Montenegro'), ('num_president', '2')]
[('rcountry', 'President of the Republic of Namibia'), ('countryname', 'Namibia'), ('num_president', '3')]
[('rcountry', 'Great President of the Republic of China'), ('countryname', 'Republic of China 1912–1949'), ('num_president', '3')]
[('rcountry', 'President of Uzbekistan'), ('countryname', 'Uzbekistan'), (

20

# Workload 3

In [ ]:
# 3. When was the first president of each country born? How many presidents each country had?

In [ ]:
# Analyzing the first question, we can start finding all the republics

In [92]:
queryString = """
SELECT ?rc ?republicountry
WHERE { 

?rc wdt:P279 wd:Q248577  . 

?rc <http://schema.org/name> ?republicountry .
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('rc', 'http://www.wikidata.org/entity/Q14946265'), ('republicountry', 'President of Bulgaria')]
[('rc', 'http://www.wikidata.org/entity/Q1054799'), ('republicountry', 'President of the Republic of Poland')]
[('rc', 'http://www.wikidata.org/entity/Q1625960'), ('republicountry', 'President of Albania')]
[('rc', 'http://www.wikidata.org/entity/Q18204367'), ('republicountry', 'President of Uganda')]
[('rc', 'http://www.wikidata.org/entity/Q19161338'), ('republicountry', 'President of Madagascar')]


5

In [ ]:
# Each republic is linked to a Country.. so i find it

In [94]:
queryString = """ 
SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('rc', 'http://www.wikidata.org/entity/Q2630370'), ('republicountry', 'President of Transnistria'), ('country', 'http://www.wikidata.org/entity/Q907112'), ('countryname', 'Transnistria')]
[('rc', 'http://www.wikidata.org/entity/Q2662118'), ('republicountry', 'President of the Sahrawi Arab Democratic Republic'), ('country', 'http://www.wikidata.org/entity/Q40362'), ('countryname', 'Sahrawi Arab Democratic Republic')]
[('rc', 'http://www.wikidata.org/entity/Q15921518'), ('republicountry', 'President of the Gambia'), ('country', 'http://www.wikidata.org/entity/Q1005'), ('countryname', 'The Gambia')]
[('rc', 'http://www.wikidata.org/entity/Q28015517'), ('republicountry', 'president of Guinea'), ('country', 'http://www.wikidata.org/entity/Q1006'), ('countryname', 'Guinea')]
[('rc', 'http://www.wikidata.org/entity/Q19161338'), ('republicountry', 'President of Madagascar'), ('country', 'http://www.wikidata.org/entity/Q1019'), ('countryname', 'Madagascar')]


5

In [ ]:
# Now to find first president born for each country, I look for the presidents of each country and his dateofbirth
# (using IRI wdt:P569 found above in the Section 1)

In [152]:
queryString = """ 
SELECT DISTINCT ?rc ?republicountry ?country ?countryname  ?dateofbirth
WHERE { 



?pr wdt:P39 ?rc ;         # ?pr = president
    wdt:P31  wd:Q5   .

OPTIONAL {          
?pr wdt:P569 ?dateofbirth 
} .

{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}

    
?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .

#?pr <http://schema.org/name> ?prname .
} 
#GROUP BY (?country)
ORDER BY  (?dateofbirth) 
LIMIT 20
"""

print("Results")
run_query(queryString)

Results
[('rc', 'http://www.wikidata.org/entity/Q878151'), ('republicountry', 'President of El Salvador'), ('country', 'http://www.wikidata.org/entity/Q792'), ('countryname', 'El Salvador')]
[('rc', 'http://www.wikidata.org/entity/Q13341442'), ('republicountry', 'President of Honduras'), ('country', 'http://www.wikidata.org/entity/Q783'), ('countryname', 'Honduras')]
[('rc', 'http://www.wikidata.org/entity/Q39021'), ('republicountry', 'President of Singapore'), ('country', 'http://www.wikidata.org/entity/Q334'), ('countryname', 'Singapore')]
[('rc', 'http://www.wikidata.org/entity/Q34366203'), ('republicountry', 'President of the Federal Republic of Central America'), ('country', 'http://www.wikidata.org/entity/Q190025'), ('countryname', 'Federal Republic of Central America')]
[('rc', 'http://www.wikidata.org/entity/Q153756'), ('republicountry', 'President of Nepal'), ('country', 'http://www.wikidata.org/entity/Q837'), ('countryname', 'Nepal')]
[('rc', 'http://www.wikidata.org/entity/Q

20

In [ ]:
# Now I look for the first president for each country, selecting the min date of birth for each country

In [153]:
queryString = """ 
SELECT DISTINCT  ?republicountry  ?countryname  (min(?dateofbirth) as ?dbirth)
WHERE { 



?pr wdt:P39 ?rc ;         # ?pr = president
    wdt:P569 ?dateofbirth ;
    wdt:P31  wd:Q5        .

{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}

#?pr <http://schema.org/name> ?prname .
} 

#ORDER BY asc  (?dateofbirth) 
GROUP BY ?republicountry ?countryname
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('republicountry', 'President of Portugal'), ('countryname', 'Portugal'), ('dbirth', '1840-06-08T00:00:00Z')]
[('republicountry', 'President of Cuba'), ('countryname', 'Cuba'), ('dbirth', '1819-04-18T00:00:00Z')]
[('republicountry', 'President of Pakistan'), ('countryname', 'Pakistan'), ('dbirth', '1899-11-13T00:00:00Z')]
[('republicountry', 'President of Austria'), ('countryname', 'Austria'), ('dbirth', '1858-08-15T00:00:00Z')]
[('republicountry', 'President of Mongolia'), ('countryname', 'Mongolia'), ('dbirth', '1881-09-11T00:00:00Z')]
[('republicountry', 'President of Transnistria'), ('countryname', 'Transnistria'), ('dbirth', '1941-10-23T00:00:00Z')]
[('republicountry', 'President of Iraq'), ('countryname', 'Iraq'), ('dbirth', '1870-07-30T00:00:00Z')]
[('republicountry', 'President of the Democratic Republic of the Congo'), ('countryname', 'Democratic Republic of the Congo'), ('dbirth', '1910-01-01T00:00:00Z')]
[('republicountry', 'President of Comoros'), ('countryname', '

10

In [68]:
# I find how many presidents has each country

In [154]:
queryString = """ 
SELECT DISTINCT  ?republicountry  ?countryname  count(distinct ?pr)
WHERE { 



?pr wdt:P39 ?rc ;        # ?pr = president
    wdt:P31  wd:Q5        .
  

{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}

#?pr <http://schema.org/name> ?prname .
} 


GROUP BY ?republicountry ?countryname
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('republicountry', 'President of Portugal'), ('countryname', 'Portugal'), ('callret-2', '19')]
[('republicountry', 'President of Cuba'), ('countryname', 'Cuba'), ('callret-2', '30')]
[('republicountry', 'President of Pakistan'), ('countryname', 'Pakistan'), ('callret-2', '15')]
[('republicountry', 'President of Austria'), ('countryname', 'Austria'), ('callret-2', '12')]
[('republicountry', 'President of Mongolia'), ('countryname', 'Mongolia'), ('callret-2', '14')]
[('republicountry', 'President of Transnistria'), ('countryname', 'Transnistria'), ('callret-2', '5')]
[('republicountry', 'President of Iraq'), ('countryname', 'Iraq'), ('callret-2', '10')]
[('republicountry', 'President of the Democratic Republic of the Congo'), ('countryname', 'Democratic Republic of the Congo'), ('callret-2', '6')]
[('republicountry', 'President of Comoros'), ('countryname', 'Comoros'), ('callret-2', '17')]
[('republicountry', 'President of Nepal'), ('countryname', 'Nepal'), ('callret-2', '3')]


10

In [ ]:
# When was the first president of each country born? How many presidents each country had?

In [155]:
queryString = """ 
SELECT DISTINCT  ?republicountry  ?countryname (count(?pr) as ?numpresidents) (min(?dateofbirth) as ?datebirth_first_president)
WHERE { 



?pr wdt:P39 ?rc ;         # ?pr = president
    wdt:P569 ?dateofbirth ;
    wdt:P31  wd:Q5        .

{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}

#?pr <http://schema.org/name> ?prname .
} 

#ORDER BY asc  (?dateofbirth) 
GROUP BY ?republicountry ?countryname
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('republicountry', 'President of Portugal'), ('countryname', 'Portugal'), ('numpresidents', '21'), ('datebirth_first_president', '1840-06-08T00:00:00Z')]
[('republicountry', 'President of Cuba'), ('countryname', 'Cuba'), ('numpresidents', '30'), ('datebirth_first_president', '1819-04-18T00:00:00Z')]
[('republicountry', 'President of Pakistan'), ('countryname', 'Pakistan'), ('numpresidents', '15'), ('datebirth_first_president', '1899-11-13T00:00:00Z')]
[('republicountry', 'President of Austria'), ('countryname', 'Austria'), ('numpresidents', '12'), ('datebirth_first_president', '1858-08-15T00:00:00Z')]
[('republicountry', 'President of Mongolia'), ('countryname', 'Mongolia'), ('numpresidents', '15'), ('datebirth_first_president', '1881-09-11T00:00:00Z')]
[('republicountry', 'President of Transnistria'), ('countryname', 'Transnistria'), ('numpresidents', '5'), ('datebirth_first_president', '1941-10-23T00:00:00Z')]
[('republicountry', 'President of Iraq'), ('countryname', 'Iraq')

10

In [ ]:
#
#

# Workload 4

In [ ]:
# 4. Is there a country that had at some point a woman as a president?

In [76]:
# GET from above: 
# sex or gender --> wdt:P21

In [ ]:
# I look for all the president of sex female

In [156]:
queryString = """
SELECT DISTINCT ?president ?presidentname ?sexorgender 
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577 ;         
           wdt:P21  ?sex         ;
           wdt:P31  wd:Q5        .
          
           

?president <http://schema.org/name> ?presidentname .
?sex <http://schema.org/name> ?sexorgender .
 
 FILTER REGEX(?sexorgender, "female")
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('president', 'http://www.wikidata.org/entity/Q1480'), ('presidentname', 'Corazon Aquino'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q57321'), ('presidentname', 'Laura Chinchilla'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q57379'), ('presidentname', 'Dalia Grybauskaitė'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q123665'), ('presidentname', 'Gloria Macapagal-Arroyo'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q207432'), ('presidentname', 'Chandrika Kumaratunga'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q467838'), ('presidentname', 'Ivy Matsepe-Casaburri'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q233984'), ('presidentname', 'Tsai Ing-wen'), ('sexorgender', 'female')]
[('president', 'http://www.wikidata.org/entity/Q40649'), ('presidentname', 'Cristina Fernández de Kirchner'), ('sex

10

In [80]:
# I want to count women out of curiosity

In [157]:
queryString = """
SELECT DISTINCT count(?president) as ?numero_di_presidenti_donne  
WHERE { 

?president wdt:P39/wdt:P279* wd:Q248577 ;         
           wdt:P21  ?sex         ;
           wdt:P31  wd:Q5        .
          
?president <http://schema.org/name> ?presidentname .
?sex <http://schema.org/name> ?sexorgender .
 
 FILTER REGEX(?sexorgender, "female")
} 
LIMIT 10

"""

print("Results")
run_query(queryString)

Results
[('numero_di_presidenti_donne', '46')]


1

In [89]:
# Now i want to show the presidents female and the their name republic

In [158]:
queryString = """ 
SELECT  DISTINCT ?republicountry  ?countryname  ?prname ?sexorgender
WHERE { 



?pr wdt:P39 ?rc ;      # ?pr = president
    wdt:P21  ?sex ;
    wdt:P31  wd:Q5        .


{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}


?pr <http://schema.org/name> ?prname .
?sex <http://schema.org/name> ?sexorgender .

 FILTER REGEX(?sexorgender, "female")

} 

LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('republicountry', 'President of Mauritius'), ('countryname', 'Mauritius'), ('prname', 'Ameenah Gurib-Fakim'), ('sexorgender', 'female')]
[('republicountry', 'President of Mauritius'), ('countryname', 'Mauritius'), ('prname', 'Monique Ohsan Bellepeau'), ('sexorgender', 'female')]
[('republicountry', 'President of Ethiopia'), ('countryname', 'Ethiopia'), ('prname', 'Sahle-Work Zewde'), ('sexorgender', 'female')]
[('republicountry', 'President of Kosovo'), ('countryname', 'Kosovo'), ('prname', 'Vjosa Osmani'), ('sexorgender', 'female')]
[('republicountry', 'President of Kosovo'), ('countryname', 'Kosovo'), ('prname', 'Atifete Jahjaga'), ('sexorgender', 'female')]
[('republicountry', 'President of Brazil'), ('countryname', 'Brazil'), ('prname', 'Dilma Rousseff'), ('sexorgender', 'female')]
[('republicountry', 'President of Iceland'), ('countryname', 'Iceland'), ('prname', 'Vigdís Finnbogadóttir'), ('sexorgender', 'female')]
[('republicountry', 'President of Estonia'), ('countryna

10

In [ ]:
#
#

# Workload 5

## Workload 5.1

In [ ]:
# 5.1 What are the top-5 countries for number of presidents? Which countries had the least?

In [161]:
queryString = """ 
SELECT DISTINCT ?countryname ?republicountry   (count( distinct ?pr) as ?numpresidents)
WHERE { 

?pr wdt:P39 ?rc ;        # ?pr = president
    wdt:P31  wd:Q5        .
  
{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}

} 

ORDER BY desc (?numpresidents) 
LIMIT 5
"""

print("Results")
run_query(queryString)

Results
[('countryname', 'Honduras'), ('republicountry', 'President of Honduras'), ('numpresidents', '83')]
[('countryname', 'Colombia'), ('republicountry', 'President of Colombia'), ('numpresidents', '60')]
[('countryname', 'Argentina'), ('republicountry', 'President of Argentina'), ('numpresidents', '59')]
[('countryname', 'Mexico'), ('republicountry', 'President of Mexico'), ('numpresidents', '59')]
[('countryname', 'Venezuela'), ('republicountry', 'President of Venezuela'), ('numpresidents', '58')]


5

In [119]:
#
#

## Workload 5.2

In [120]:
# 5.2 For how many presidents we know a date of death in each country?

In [ ]:
# GET:  wdt:P570 (date death) from above

In [163]:
queryString = """ 
SELECT DISTINCT ?countryname ?republicountry   (count( distinct ?pr) as ?numpresidents)
WHERE { 

?pr wdt:P39 ?rc ;   # ?pr = president
    wdt:P570 ?dateofdeath ;
    wdt:P31  wd:Q5        .
  
{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}

} 

ORDER BY desc (?numpresidents) 
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('countryname', 'Honduras'), ('republicountry', 'President of Honduras'), ('numpresidents', '72')]
[('countryname', 'Venezuela'), ('republicountry', 'President of Venezuela'), ('numpresidents', '55')]
[('countryname', 'Colombia'), ('republicountry', 'President of Colombia'), ('numpresidents', '54')]
[('countryname', 'Mexico'), ('republicountry', 'President of Mexico'), ('numpresidents', '52')]
[('countryname', 'Argentina'), ('republicountry', 'President of Argentina'), ('numpresidents', '51')]
[('countryname', 'Paraguay'), ('republicountry', 'President of Paraguay'), ('numpresidents', '49')]
[('countryname', 'Haiti'), ('republicountry', 'President of Haiti'), ('numpresidents', '48')]
[('countryname', 'Peru'), ('republicountry', 'President of Peru'), ('numpresidents', '44')]
[('countryname', 'Guatemala'), ('republicountry', 'President of the Republic of Guatemala'), ('numpresidents', '43')]
[('countryname', 'United States of America'), ('republicountry', 'President of the Unite

10

## Workload 5.3

In [ ]:
# 5.3 Which are the professions of different presidents? How many presidents had a specific profession?

In [14]:
# GET from above: 
# occupation --> wdt:P106

In [ ]:
# Find the number of different professions of different presidents

In [13]:
queryString = """ 
SELECT DISTINCT  count(?ocname) as ?num  #?countryname ?republicountry ?pr ?prname
WHERE { 

?pr wdt:P39 ?rc ;  # ?pr = president
    wdt:P31 wd:Q5   ;
    wdt:P106 ?oc .
    
  
{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}
?pr <http://schema.org/name> ?prname .
?oc <http://schema.org/name> ?ocname .


} 


LIMIT 200
"""

print("Results")
run_query(queryString)

Results
[('num', '3305')]


1

In [ ]:
# Find all the professions of different presidents

In [15]:
queryString = """ 
SELECT DISTINCT  ?ocname  #?countryname ?republicountry ?pr ?prname
WHERE { 

?pr wdt:P39 ?rc ;  # ?pr = president
    wdt:P31 wd:Q5   ;
    wdt:P106 ?oc .
    
  
{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}
?pr <http://schema.org/name> ?prname .
?oc <http://schema.org/name> ?ocname .


} 


LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('ocname', 'herder')]
[('ocname', 'diplomat')]
[('ocname', 'politician')]
[('ocname', 'serial rapist')]
[('ocname', 'military officer')]
[('ocname', 'treasurer')]
[('ocname', 'writer')]
[('ocname', 'statesperson')]
[('ocname', 'disc jockey')]
[('ocname', 'economist')]


10

In [ ]:
# I find the first 10 profession (ordered by number of president with that profession) and their number of president

In [17]:
queryString = """ 
SELECT DISTINCT  ?ocname (count(?pr) as ?numpr)
WHERE { 

?pr wdt:P39 ?rc ;  # ?pr = president
    wdt:P31 wd:Q5   ;
    wdt:P106 ?oc .
    
  
{
 SELECT ?rc ?republicountry ?country ?countryname
WHERE { 

?rc wdt:P279 wd:Q248577  ; 
    wdt:P17 ?country .

?rc <http://schema.org/name> ?republicountry .

?country <http://schema.org/name> ?countryname .
} 
}
?pr <http://schema.org/name> ?prname .
?oc <http://schema.org/name> ?ocname .


} 

GROUP BY ?ocname
ORDER BY DESC (?numpr)
LIMIT 10
"""

print("Results")
run_query(queryString)

Results
[('ocname', 'politician'), ('numpr', '1482')]
[('ocname', 'lawyer'), ('numpr', '340')]
[('ocname', 'diplomat'), ('numpr', '205')]
[('ocname', 'military personnel'), ('numpr', '194')]
[('ocname', 'journalist'), ('numpr', '82')]
[('ocname', 'economist'), ('numpr', '74')]
[('ocname', 'writer'), ('numpr', '72')]
[('ocname', 'judge'), ('numpr', '66')]
[('ocname', 'university teacher'), ('numpr', '64')]
[('ocname', 'military officer'), ('numpr', '61')]


10